In [1]:
from data_generator import *

## Example of parsing active constraints: DC-OPF

In [2]:
org_dir = os.getcwd()
os.chdir('./matpower7.1/')

In [3]:
eng = matlab.engine.start_matlab()

In [4]:
case_name = "pglib_opf_case14_ieee.m"

In [5]:
data = eng.dc_opf_solver(case_name, 0.05)
eng.quit()
os.chdir(org_dir)

In [6]:
data = standardize_data_type(data=data, d_type="float32")
data

{'bus_info': {'p_d': array([[ 0.       ],
         [23.689766 ],
         [83.56083  ],
         [49.860596 ],
         [ 7.721131 ],
         [10.467694 ],
         [ 0.       ],
         [ 0.       ],
         [34.778137 ],
         [10.246246 ],
         [ 3.2637699],
         [ 7.0256515],
         [13.989648 ],
         [14.8530245]], dtype=float32),
  'bus_idx': array([[ 1.],
         [ 2.],
         [ 3.],
         [ 4.],
         [ 5.],
         [ 6.],
         [ 7.],
         [ 8.],
         [ 9.],
         [10.],
         [11.],
         [12.],
         [13.],
         [14.]], dtype=float32)},
 'gen_info': {'p_g': array([[259.45648],
         [  0.     ],
         [  0.     ],
         [  0.     ],
         [  0.     ]], dtype=float32),
  'p_g_lim': array([[340.,   0.],
         [ 59.,   0.],
         [  0.,   0.],
         [  0.,   0.],
         [  0.,   0.]], dtype=float32),
  'gen2bus': array([[1.],
         [2.],
         [3.],
         [6.],
         [8.]], dtype=float32

### an example of feature: x

In [7]:
data["w_info"]["w"]

array([[  0.        ,   1.9897653 , -10.639169  ,   2.0605943 ,
          0.12113079,  -0.73230547,   0.        ,   0.        ,
          5.2781353 ,   1.2462467 ,  -0.23623021,   0.92565167,
          0.48964787,  -0.04697588]], dtype=float32)

### an example of lable: y

In [8]:
gen_active = get_active_gen_constraints(data)
gen_active

array([[0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

In [9]:
flow_active = get_active_flow_constraints(data)
flow_active

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

In [10]:
active_constraints = merge_active_constraints(gen_active, flow_active)
active_constraints

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

### a graph representaion of the power system

## Build Datasets

### example of creating a dataset
- **x**: uncertainty realization as a feature input
- **y**: active constraints index as a label output

In [14]:
dataset = create_dataset(test_case=case_name, dataset_size=3, std_scaler=0.03)
dataset

> creating dataset with pglib_opf_case24_ieee_rts.m


100%|██████████| 3/3 [00:03<00:00,  1.28s/it]


{'x': array([[  1.7420416 ,   5.3366055 , -12.197773  ,   1.9140248 ,
           0.67897   ,  -5.335368  ,  -1.6259701 ,   1.7576635 ,
          18.786585  ,  16.201206  ,   0.        ,   0.        ,
           5.7669635 ,  -0.36697936,   6.797205  ,  -0.61489815,
           0.        ,  14.882079  ,   7.6510572 ,   5.442019  ,
           0.        ,   0.        ,   0.        ,   0.        ],
        [  1.5840158 ,   3.0109568 ,   3.9251797 ,  -0.6736389 ,
           0.6259462 ,  -3.2121139 ,   3.3314836 ,  -5.8844695 ,
          -5.61157   ,  -4.7355676 ,   0.        ,   0.        ,
           2.5852647 ,  -4.393683  ,  13.031539  ,  -5.134549  ,
           0.        ,  -2.412056  ,   1.7332926 ,   1.201377  ,
           0.        ,   0.        ,   0.        ,   0.        ],
        [  3.5421808 ,   3.2279854 ,  -4.6637254 ,   0.17173718,
          -2.5860693 ,  -4.543083  ,  -0.02568498,   7.8623934 ,
          -4.040746  ,   2.172566  ,   0.        ,   0.        ,
          -8.65806

### Build datasets

In [17]:
test_cases = [
    "pglib_opf_case24_ieee_rts.m",
    "pglib_opf_case30_ieee.m",
    "pglib_opf_case39_epri.m",
    "pglib_opf_case57_ieee.m",
    "pglib_opf_case73_ieee_rts.m",
    "pglib_opf_case118_ieee.m",
    "pglib_opf_case162_ieee_dtc.m",
    "pglib_opf_case300_ieee.m",
    "pglib_opf_case1888_rte.m"
]

dataset_size = 10
std_scaler = 0.05

In [18]:
build_datasets(test_cases, dataset_size)

> creating dataset with pglib_opf_case24_ieee_rts.m


100%|██████████| 10/10 [00:04<00:00,  2.28it/s]


> creating dataset with pglib_opf_case30_ieee.m


100%|██████████| 10/10 [00:04<00:00,  2.30it/s]


> creating dataset with pglib_opf_case39_epri.m


...

100%|██████████| 10/10 [00:05<00:00,  1.75it/s]


> creating dataset with pglib_opf_case300_ieee.m


100%|██████████| 10/10 [00:05<00:00,  1.74it/s]


> creating dataset with pglib_opf_case1888_rte.m


100%|██████████| 10/10 [00:08<00:00,  1.15it/s]


### Check datasets

In [26]:
infile = open('./../../data/' + str(dataset_size) +'/case30.pickle','rb')
dataset = pickle.load(infile)
infile.close()

In [27]:
dataset

{'x': array([[ 0.        , -0.74674264, -0.07695867, -0.1845657 ,  0.        ,
          0.        ,  0.22243033, -0.67943549,  0.        , -0.29780386,
          0.        , -0.08112621,  0.        ,  0.0581917 , -0.21276046,
         -0.00315539, -0.04451734,  0.0602599 ,  0.31158072,  0.07321204,
         -0.45341773,  0.        , -0.11655524, -0.29062369,  0.        ,
          0.16092618,  0.        ,  0.        , -0.01624208,  0.35531925],
        [ 0.        ,  0.02119491,  0.03978195,  0.25093913,  0.        ,
          0.        , -1.02024777, -0.66807165,  0.        ,  0.40897956,
          0.        ,  0.2513538 ,  0.        ,  0.16528154, -0.18815291,
         -0.14723824, -0.3840415 ,  0.04686662, -0.05055192, -0.01293953,
          0.74513783,  0.        ,  0.01898986,  0.41438946,  0.        ,
          0.07314556,  0.        ,  0.        ,  0.01552825, -0.37073837],
        [ 0.        ,  0.06827344,  0.05200229,  0.58949201,  0.        ,
          0.        , -0.056426